# Descarga del coprus y PREPROCESAMIENTO 

In [5]:
import json
import random
import os
import pandas as pd

In [ ]:
corpus_path = r"C:\Users\roble\OneDrive\Documentos\GitHub\Recuperacion\ir25a-main\ir25a-main\data\EXAMEN\arxiv-metadata-oai-snapshot.json"
base_path = r"C:\Users\roble\OneDrive\Documentos\GitHub\Recuperacion\ir25a-main\ir25a-main\data\EXAMEN"
output_jsonl = os.path.join(base_path, "corpus_1_percent.jsonl")
output_json = os.path.join(base_path, "corpus_1_percent.json")

if not os.path.exists(corpus_path):
    raise FileNotFoundError(f"No se encontró el archivo en la ruta: {corpus_path}")

# --- Leer corpus como JSON Lines ---
documents = []
with open(corpus_path, 'r', encoding='utf-8') as file:
    for i, line in enumerate(file, 1):
        try:
            doc = json.loads(line)
            documents.append(doc)
            if i % 10000 == 0:
                print(f"{i} documentos cargados...")
        except json.JSONDecodeError as e:
            if i <= 5:
                print(f"Error en línea {i}: {e} (parcial: {line[:80]}...)")
            continue

total_docs = len(documents)
print(f"Total documentos cargados: {total_docs}")

# --- Muestreo aleatorio del 1% del corpus ---
sample_size = max(1, int(total_docs * 0.01))
random.seed(42)
sample_docs = random.sample(documents, sample_size)
print(f"Muestra (1%): {sample_size} documentos")

# --- Guardar la muestra en JSON Lines y JSON ---
with open(output_jsonl, 'w', encoding='utf-8') as file:
    for doc in sample_docs:
        json.dump(doc, file, ensure_ascii=False)
        file.write('\n')
print(f"Guardado: {output_jsonl}")

with open(output_json, 'w', encoding='utf-8') as file:
    json.dump(sample_docs, file, indent=2, ensure_ascii=False)
print(f"Guardado: {output_json}")

# --- Mostrar algunos ejemplos para verificar ---
if sample_docs:
    print("\nCampos del primer documento:", list(sample_docs[0].keys()))
    print("\nEjemplo de títulos:")
    for i, doc in enumerate(sample_docs[:3]):
        print(f"  {i+1}. {doc.get('title', '')[:100]}...")
    print("\nEjemplo de abstracts:")
    for i, doc in enumerate(sample_docs[:2]):
        print(f"  {i+1}. {doc.get('abstract', '')[:150]}...")


10000 documentos cargados...
20000 documentos cargados...
30000 documentos cargados...
40000 documentos cargados...
50000 documentos cargados...
60000 documentos cargados...
70000 documentos cargados...
80000 documentos cargados...
90000 documentos cargados...
100000 documentos cargados...
110000 documentos cargados...
120000 documentos cargados...
130000 documentos cargados...
140000 documentos cargados...
150000 documentos cargados...
160000 documentos cargados...
170000 documentos cargados...
180000 documentos cargados...
190000 documentos cargados...
200000 documentos cargados...
210000 documentos cargados...
220000 documentos cargados...
230000 documentos cargados...
240000 documentos cargados...
250000 documentos cargados...
260000 documentos cargados...
270000 documentos cargados...
280000 documentos cargados...
290000 documentos cargados...
300000 documentos cargados...
310000 documentos cargados...
320000 documentos cargados...
330000 documentos cargados...
340000 documentos c

In [ ]:
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
import pandas as pd

# --- Prepara stopwords solo una vez ---
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    if not text or pd.isna(text):
        return []
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    return tokens

# --- Ruta del corpus de muestra ---
sample_path = r"C:\Users\roble\OneDrive\Documentos\GitHub\Recuperacion\ir25a-main\ir25a-main\data\EXAMEN\corpus_1_percent.json"

if not os.path.exists(sample_path):
    raise FileNotFoundError(f"No se encontró el archivo de muestra en: {sample_path}")

# --- Cargar documentos de muestra ---
with open(sample_path, 'r', encoding='utf-8') as file:
    sample_docs = json.load(file)

print(f"Documentos cargados: {len(sample_docs)}")

# --- DataFrame ORIGINAL (sin limpiar) ---
df_original = pd.DataFrame([{
    'id': doc.get('id', ''),
    'title': doc.get('title', ''),
    'abstract': doc.get('abstract', '')
} for doc in sample_docs])

print("\nDataFrame original (sin limpiar):")
display(df_original.head(3))

# --- Preprocesamiento y creación de texto limpio ---
def tokens_to_text(tokens):
    return ' '.join(tokens)

processed_corpus = []
for i, doc in enumerate(sample_docs):
    title = doc.get('title', '')
    abstract = doc.get('abstract', '')
    title_tokens = preprocess_text(title)
    abstract_tokens = preprocess_text(abstract)
    combined_tokens = title_tokens + abstract_tokens
    processed_corpus.append({
        'id': doc.get('id', f'doc_{i}'),
        'original_title': title,
        'original_abstract': abstract,
        'text_clean': tokens_to_text(combined_tokens)
    })

df_clean = pd.DataFrame(processed_corpus)

print("\nDataFrame con texto limpio (solo las 3 primeras filas):")
display(df_clean[['id', 'original_title', 'original_abstract', 'text_clean']].head(3))

# --- Estadísticas útiles ---
print(f"\nTotal de documentos procesados: {len(df_clean)}")
print(f"Promedio de tokens por documento: {df_clean['text_clean'].str.split().apply(len).mean():.2f}")


Documentos cargados: 27923

DataFrame original (sin limpiar):


,id,title,abstract
0,math/0007070,Universal homotopy theories,"Given a small category C, we show that there..."
1,1310.1953,The dynamics of correlated novelties,One new thing often leads to another. Such c...
2,0901.3660,Pomeron loops in the perturbative QCD with Lar...,The lowest order pomeron loop is calculated ...



DataFrame con texto limpio (solo las 3 primeras filas):


,id,original_title,original_abstract,text_clean
0,math/0007070,Universal homotopy theories,"Given a small category C, we show that there...",universal homotopy theories given small catego...
1,1310.1953,The dynamics of correlated novelties,One new thing often leads to another. Such c...,dynamics correlated novelties one new thing of...
2,0901.3660,Pomeron loops in the perturbative QCD with Lar...,The lowest order pomeron loop is calculated ...,pomeron loops perturbative qcd large lowest or...



Total de documentos procesados: 27923
Promedio de tokens por documento: 95.43


# Indexacion

## TFIDF


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [3]:
# --- Ruta del corpus preprocesado ---
preprocessed_path = r"C:\Users\roble\OneDrive\Documentos\GitHub\Recuperacion\ir25a-main\ir25a-main\data\EXAMEN\corpus_preprocessed.json"

if not os.path.exists(preprocessed_path):
    raise FileNotFoundError(f"No se encontró el archivo: {preprocessed_path}")

# --- Cargar corpus preprocesado ---
with open(preprocessed_path, 'r', encoding='utf-8') as file:
    processed_corpus = json.load(file)

# --- Preparar textos para TF-IDF ---
documents = [' '.join(doc['combined_tokens']) for doc in processed_corpus]
doc_ids = [doc['id'] for doc in processed_corpus]

print(f"Documentos a indexar: {len(documents)}")

# --- Crear y entrenar vectorizador TF-IDF ---
tfidf_vectorizer = TfidfVectorizer(
    lowercase=False,        # Ya está preprocesado
    token_pattern=r'(?u)\b\w+\b',
    max_features=5000
)
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

print(f"TF-IDF listo: documentos={tfidf_matrix.shape[0]}, términos únicos={tfidf_matrix.shape[1]}")
print(f"Densidad de la matriz: {tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1]):.4f}")
print("Primeros 10 términos del vocabulario:", tfidf_vectorizer.get_feature_names_out()[:10])


Documentos a indexar: 27923
TF-IDF listo: documentos=27923, términos únicos=5000
Densidad de la matriz: 0.0119
Primeros 10 términos del vocabulario: ['abelian' 'abilities' 'ability' 'ablation' 'able' 'absence' 'absent'
 'absolute' 'absorbing' 'absorption']


## BM25

In [6]:
from rank_bm25 import BM25Okapi

# Usar los mismos documentos tokenizados del TF-IDF
tokenized_docs = []
for doc in processed_corpus:
    tokenized_docs.append(doc['combined_tokens'])

print(f"Creando índice BM25 para {len(tokenized_docs)} documentos...")

# Crear índice BM25
bm25 = BM25Okapi(tokenized_docs)

print(f"Índice BM25 creado:")
print(f"- Documentos: {len(tokenized_docs)}")
print(f"- Vocabulario: {len(bm25.doc_freqs)} términos únicos")

Creando índice BM25 para 27923 documentos...
Índice BM25 creado:
- Documentos: 27923
- Vocabulario: 27923 términos únicos


## Embeddings

In [7]:
from sentence_transformers import SentenceTransformer
import faiss

In [8]:
# Cargar modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Preparar textos para embeddings (usar documentos preprocesados)
texts = []
for doc in processed_corpus:
    # Combinar título y abstract originales
    text = doc['original_title'] + ' ' + doc['original_abstract']
    texts.append(text)

print(f"Generando embeddings para {len(texts)} documentos...")

# Generar embeddings
embeddings = model.encode(texts, show_progress_bar=True)

print(f"Embeddings generados:")
print(f"- Dimensión: {embeddings.shape[1]}")
print(f"- Documentos: {embeddings.shape[0]}")

# Crear índice FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product para similitud coseno

# Normalizar embeddings para similitud coseno
faiss.normalize_L2(embeddings)

# Añadir embeddings al índice
index.add(embeddings.astype('float32'))

print(f"\nÍndice FAISS creado:")
print(f"- Total vectores indexados: {index.ntotal}")
print(f"- Dimensión: {dimension}")

Generando embeddings para 27923 documentos...


Batches:   0%|          | 0/873 [00:00<?, ?it/s]

Embeddings generados:
- Dimensión: 384
- Documentos: 27923

Índice FAISS creado:
- Total vectores indexados: 27923
- Dimensión: 384


# Recuperacion

## TF-IDF

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search_tfidf(query, top_k=10):
    query_vec = tfidf_vectorizer.transform([query])
    scores = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_indices = np.argsort(scores)[::-1][:top_k]
    return [{'id': doc_ids[i], 'score': scores[i]} for i in top_indices]


## BM25


In [14]:
from rank_bm25 import BM25Okapi

def search_bm25(query, top_k=10):
    tokenized_query = query.lower().split()
    scores = bm25.get_scores(tokenized_query)
    top_indices = np.argsort(scores)[::-1][:top_k]
    return [{'id': doc_ids[i], 'score': scores[i]} for i in top_indices]

## FAIISS

In [19]:
def search_faiss(query, top_k=10):
    query_emb = model.encode([query]).astype('float32')  # <- usas 'model' no 'sentence_model'
    faiss.normalize_L2(query_emb)  # <- porque tu índice está normalizado
    distances, indices = index.search(query_emb, top_k)  # <- usas 'index' no 'faiss_index'
    return [{'id': doc_ids[i], 'distance': distances[0][rank]} for rank, i in enumerate(indices[0])]


### DataFrame comparativo de resultados

In [20]:
import pandas as pd

# --- 1. Leer queries ---
queries_path = r"C:\Users\roble\OneDrive\Documentos\GitHub\Recuperacion\ir25a-main\ir25a-main\data\queries\queries.txt"
with open(queries_path, 'r', encoding='utf-8') as f:
    queries = [line.strip() for line in f if line.strip()]

# --- 2. Buscar Top 10 IDs para cada query con cada modelo ---
tfidf_top10_by_query = []
bm25_top10_by_query = []
faiss_top10_by_query = []

for query in queries:
    tfidf_result = search_tfidf(query, top_k=10)
    bm25_result = search_bm25(query, top_k=10)
    faiss_result = search_faiss(query, top_k=10)

    tfidf_top10_by_query.append([doc['id'] for doc in tfidf_result])
    bm25_top10_by_query.append([doc['id'] for doc in bm25_result])
    faiss_top10_by_query.append([doc['id'] for doc in faiss_result])

# --- 3. Comparar documentos comunes entre modelos ---
comparaciones = []
for i, query in enumerate(queries):
    tfidf_set = set(tfidf_top10_by_query[i])
    bm25_set = set(bm25_top10_by_query[i])
    faiss_set = set(faiss_top10_by_query[i])

    comunes_tfidf_bm25 = len(tfidf_set & bm25_set)
    comunes_tfidf_faiss = len(tfidf_set & faiss_set)
    comunes_bm25_faiss = len(bm25_set & faiss_set)

    comparaciones.append({
        'Query': query,
        'TF-IDF ∩ BM25': comunes_tfidf_bm25,
        'TF-IDF ∩ FAISS': comunes_tfidf_faiss,
        'BM25 ∩ FAISS': comunes_bm25_faiss
    })

# --- 4. Mostrar resultados ---
df_comparacion = pd.DataFrame(comparaciones)
df_comparacion


,Query,TF-IDF ∩ BM25,TF-IDF ∩ FAISS,BM25 ∩ FAISS
0,transformers in natural language processing,4,1,2
1,deep reinforcement learning algorithms,4,2,3
2,adversarial attacks in computer vision,6,2,2
3,graph neural networks applications,8,3,3
4,explainable artificial intelligence methods,5,1,1


### Análisis de intersección entre métodos de recuperación

En la comparación de los resultados obtenidos por los tres métodos de recuperación (TF-IDF, BM25 y FAISS) para las cinco consultas analizadas, se observan varios patrones interesantes:

- **TF-IDF y BM25** muestran una mayor coincidencia entre sí. En consultas como `"graph neural networks applications"` se observa una coincidencia de **8 documentos sobre 10**, lo que refleja una alta similitud en los criterios de relevancia de ambos modelos basados en estadística de términos.
  
- La intersección entre **TF-IDF y FAISS** es considerablemente más baja, con un máximo de **3 coincidencias**, lo cual es esperable ya que FAISS se basa en embeddings semánticos y no en ocurrencias de palabras clave.

- **BM25 y FAISS** también presentan coincidencias moderadas (hasta 3 documentos en común), pero en general muestran menos alineación que BM25 con TF-IDF.

- En ningún caso se presenta una coincidencia perfecta entre los tres métodos al mismo tiempo, lo que indica que **cada modelo prioriza documentos distintos** según su forma de evaluar la similitud.

Este análisis evidencia que TF-IDF y BM25, al ser métodos basados en términos, tienden a recuperar documentos similares, mientras que FAISS aporta diversidad al enfocarse en la semántica del texto. Esto justifica el uso de modelos complementarios en sistemas de recuperación híbridos.


# RAG

### OpenAI

In [ ]:
from openai import OpenAI
API_KEY = ""
client = OpenAI(api_key=API_KEY)

In [23]:
# 1. Prepara la consulta y extrae los top-3 FAISS
consulta_rag = "machine learning"
top3_docs = search_faiss(consulta_rag, top_k=3)

# 2. Encuentra los documentos completos para cada ID
contexto = ""
for i, doc_info in enumerate(top3_docs):
    doc_id = doc_info['id']
    # Buscar el documento completo en el corpus procesado
    original_doc = None
    for doc in processed_corpus:
        if doc['id'] == doc_id:
            original_doc = doc
            break
    
    if original_doc:
        contexto += f"[Doc {i+1}] ID: {doc_id}\nTítulo: {original_doc['original_title']}\nResumen: {original_doc['original_abstract']}\n\n"
    else:
        contexto += f"[Doc {i+1}] ID: {doc_id}\n(Documento completo no encontrado)\n\n"

prompt_rag = (
    f"Eres un experto en artículos científicos. Usa SOLO la información del siguiente contexto para responder "
    f"la pregunta del usuario. Si no está la respuesta, di explícitamente que no sabes.\n\n"
    f"Contexto recuperado:\n{contexto}\n"
    f"Pregunta del usuario: {consulta_rag}\n\n"
    f"Respuesta:"
)

# 3. Llama a la API de OpenAI
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Responde de forma clara y concisa, usando SOLO el contexto proporcionado."},
        {"role": "user", "content": prompt_rag}
    ],
    max_tokens=350,
    temperature=0.2
)

respuesta_rag = response.choices[0].message.content

print("=== Prompt enviado al modelo ===\n")
print(prompt_rag)
print("\n=== Respuesta generada (RAG) ===\n")
print(respuesta_rag)


=== Prompt enviado al modelo ===

Eres un experto en artículos científicos. Usa SOLO la información del siguiente contexto para responder la pregunta del usuario. Si no está la respuesta, di explícitamente que no sabes.

Contexto recuperado:
[Doc 1] ID: 1810.11772
Título: Learning with Analytical Models
Resumen:   To understand and predict the performance of scientific applications, several
analytical and machine learning approaches have been proposed, each having its
advantages and disadvantages. In this paper, we propose and validate a hybrid
approach for performance modeling and prediction, which combines analytical and
machine learning models. The proposed hybrid model aims to minimize prediction
cost while providing reasonable prediction accuracy. Our validation results
show that the hybrid model is able to learn and correct the analytical models
to better match the actual performance. Furthermore, the proposed hybrid model
improves the prediction accuracy in comparison to pure ma